## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-12-06-37-00 +0000,wsj,German 10-Year Bund Yield Likely to Stay Below...,https://www.wsj.com/finance/german-10-year-bun...
1,2026-01-12-06-06-24 +0000,bbc,"Watch: Timothée Chalamet, Teyana Taylor, Adole...",https://www.bbc.com/news/videos/ckgnr12902no?a...
2,2026-01-12-06-03-19 +0000,nyt,Mass Protests in Iran and Trump’s Threats,https://www.nytimes.com/2026/01/11/world/mass-...
3,2026-01-12-05-56-00 +0000,wsj,The criminal investigation into Federal Reserv...,https://www.wsj.com/economy/central-banking/tr...
4,2026-01-12-05-50-51 +0000,nyt,"As Death Toll Surges in Iran, Leaders Take Tou...",https://www.nytimes.com/2026/01/11/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2407/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,32
19,iran,18
268,ice,12
144,shooting,8
18,protests,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
73,2026-01-11-22-39-27 +0000,nyt,Deposed Shah’s Son Hopes Trump Will Put Iran R...,https://www.nytimes.com/2026/01/11/world/europ...,73
87,2026-01-11-21-55-03 +0000,nyt,Trump Says Cuba Will No Longer Get ‘Oil or Mon...,https://www.nytimes.com/2026/01/11/us/politics...,67
132,2026-01-11-16-29-40 +0000,cbc,Trump says no more Venezuelan oil or money to ...,https://www.cbc.ca/news/world/trump-says-no-mo...,67
59,2026-01-12-00-12-29 +0000,nyt,Thousands March in New York to Protest ICE Kil...,https://www.nytimes.com/2026/01/11/us/politics...,65
133,2026-01-11-16-28-00 +0000,wsj,South Africa Risks Angering Trump by Hosting N...,https://www.wsj.com/world/africa/south-africa-...,63


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
73,73,2026-01-11-22-39-27 +0000,nyt,Deposed Shah’s Son Hopes Trump Will Put Iran R...,https://www.nytimes.com/2026/01/11/world/europ...
3,58,2026-01-12-05-56-00 +0000,wsj,The criminal investigation into Federal Reserv...,https://www.wsj.com/economy/central-banking/tr...
74,50,2026-01-11-22-37-17 +0000,nypost,"NYC anti-ICE protest draws 2,000 in latest unr...",https://nypost.com/2026/01/11/us-news/nyc-anti...
132,35,2026-01-11-16-29-40 +0000,cbc,Trump says no more Venezuelan oil or money to ...,https://www.cbc.ca/news/world/trump-says-no-mo...
10,33,2026-01-12-05-00-04 +0000,nypost,Arkansas mother killed alongside her 6-year-ol...,https://nypost.com/2026/01/12/us-news/arkansas...
32,28,2026-01-12-02-33-58 +0000,nypost,Black New York homeowners blast Mamdani’s radi...,https://nypost.com/2026/01/11/us-news/black-ne...
172,28,2026-01-11-10-30-00 +0000,wsj,Silicon Valley has long been something of an A...,https://www.wsj.com/politics/policy/its-a-weir...
151,26,2026-01-11-13-00-00 +0000,nypost,After Marine vet Trevor Reed was freed from a ...,https://nypost.com/2026/01/11/world-news/marin...
24,24,2026-01-12-03-16-59 +0000,startribune,Minnesota’s Sun Country Airlines to be acquire...,https://www.startribune.com/sun-country-allegi...
126,24,2026-01-11-17-21-26 +0000,bbc,Thousands march and dozens arrested in Minneap...,https://www.bbc.com/news/articles/cvgpnwnqygro...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
